In [1]:
%load_ext autoreload
%autoreload 2

import os
DATA_DIR = os.path.expanduser("~/repos/data/")
# DATA_DIR = "/data1/simulations/datasets/rotors/high_temp_ML_training_data/"

FANDE_DIR = os.path.expanduser("~/repos/")
# RESULTS_DIR = os.path.expanduser("~/repos/data/results")
# os.makedirs(RESULTS_DIR, exist_ok=True)

import os
import sys
import torch
sys.path.append(FANDE_DIR + "fande/")



from ase import io

traj_295 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index="0:10")
# traj_355 = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")
# traj_295_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
# traj_355_2000K = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
# traj_295_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
# traj_355_2000K_forced = io.read(DATA_DIR+"/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")

In [9]:
init_structure = traj_295[0]
init_structure.set_pbc(False)


calc_dir = DATA_DIR + "/results/test_run/"
calculator = 'xtb'
num_instances = 2
num_steps = 10
n_beads = 6
ipi_port = 10200

input_xml_str = f"""
<simulation verbosity='high'>
  <output prefix='PREFIX'>
<!--    <properties stride='1' filename='out'>  [ step, potential{{kilocal/mol}} ] </properties>-->
    <!-- <trajectory stride='1' filename='pos'>  positions{{angstrom}}  </trajectory> -->
    <!-- <properties stride="10"> [ step, time, pot_component_raw(0), pot_component_raw(1), temperature, conserved ] </properties> -->
    <properties stride="1"> [ step, time, potential(0), temperature, conserved ] </properties>
    <trajectory stride="1" filename="pos" cell_units="angstrom"> positions{{angstrom}} </trajectory>
    <trajectory stride="1" filename="pos_centroid" cell_units="angstrom"> x_centroid{{angstrom}} </trajectory>
    <trajectory stride="1" filename="frc"> forces </trajectory>
    <checkpoint stride="1000" overwrite="False"/>
  </output>
  <total_steps> {num_steps} </total_steps>
  <ffsocket mode='inet' name='mycalc' pbc='True'>
    <address>localhost</address>
    <port>{ipi_port}</port>
  </ffsocket>
  <system>
    <initialize nbeads='{n_beads}'>
      <file mode='ase'>  init.xyz </file>
      <velocities mode="thermal" units="kelvin"> 20 </velocities>
    </initialize>
    <forces>
      <force forcefield='mycalc'> </force>
    </forces>
    <ensemble>
      <temperature units='kelvin'> 300.0 </temperature>
    </ensemble>
    <motion mode='dynamics'>
      <dynamics mode='nvt'>
        <timestep units='femtosecond'> 0.5 </timestep>
        <thermostat mode='langevin'>
          <tau units='femtosecond'>10</tau>
        </thermostat>
      </dynamics>
    </motion>
  </system>
</simulation>
"""

In [25]:
from fande import ipi
ipi.launch_server(init_structure, calc_dir, input_xml_str)
ipi.launch_drivers(init_structure, calculator, num_instances, calc_dir, ipi_port)

Launching i-pi...
bash /home/mm/repos/data/results/test_run/launch.sh
Launched! Please manually check if i-pi is running.
Starting clients with joblib...


fande.models module imported...
Icecream logger is not available
fande.models module imported...
Icecream logger is not available
fande.models module imported...
Icecream logger is not available
fande.models module imported...
fande.models module imported...
Icecream logger is not availableIcecream logger is not available

fande.models module imported...
Icecream logger is not available
Calculator is set up!
Launching client 0 at port 10200
Calculator is set up!
Launching client 2 at port 10200
Calculator is set up!
Launching client 3 at port 10200
Calculator is set up!
Launching client 5 at port 10200
Calculator is set up!
Launching client 1 at port 10200
Calculator is set up!
Launching client 4 at port 10200


          ==================== Thresholds ====================
          ==================== Thresholds ====================
          CN  :   150.00000
          CN  :   150.00000
          rep :   500.00000
          rep :   500.00000
          disp:  2500.00000
          di

[0, 0, 0, 0, 0, 0]

In [24]:
from fande import ipi
ipi.kill_all()

0

In [18]:
import os
os.getcwd()

'/home/mm/repos/rotor-gp/code'

In [12]:
ipi.exit_calculation(calc_dir)

0

In [36]:
ipi.restart(calc_dir)

0